In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# # Load the model and tokenizer from the saved directory
# # model_dir = "/kaggle/working/my_model"
# model_dir = "microsoft/Phi-3-mini-4k-instruct"
# model = AutoModelForCausalLM.from_pretrained(model_dir)
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

# # Save the model and tokenizer to the local filesystem
# model.save_pretrained("/kaggle/working/my_model")
# tokenizer.save_pretrained("/kaggle/working/my_model")

In [ ]:
# Load the model and tokenizer from the saved directory
model_dir = "/kaggle/working/my_model"
model = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
# Create a text generation pipeline
text_generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,  # Use GPU if available
    framework="pt",  # PyTorch framework
    temperature=0.5,
    do_sample=True,
    max_length=500  # Adjusted from max_new_tokens to max_length
)

# Example usage
prompt = "What happend, Once upon a time?"
generated_text = text_generator(prompt)
print(generated_text)


In [ ]:
from langchain_huggingface import HuggingFacePipeline
hf = HuggingFacePipeline(pipeline = text_generator)
hf.invoke("Hello!")

In [ ]:
from langchain.prompts import PromptTemplate

def generate_restaurant_name_and_items(cuisine):
    # Chain 1: Restaurant Name
    prompt_template_name = PromptTemplate(
        input_variables = ['cuisine'],
        template = "I want to open a new restaurant for {cuisine} cuisine. Can you suggest ONLY ONE name for my restaurant?, Give only name, nothing else!"
    )
    
    name = hf.invoke(prompt_template_name.format(cuisine = cuisine))
#     chain = prompt_template_name | hf
#     name = chain.invoke()
    
    # Chain 2: Menu items
    prompt_template_name = PromptTemplate(
        input_variables= ['restaurant_name'],
        template = "Suggest some menu items for {restaurant_name} No sentence, only name. Return it as a comma seperated string"
    )
    
    menu_items = hf.invoke(prompt_template_name.format(restaurant_name = name))
#     chain = prompt_template_name | model
#     menu_items = chain.invoke({'restaurant_name': cuisine})
    
    return {
        'restaurant_name': name,
        'menu_items': menu_items
    }

if __name__ == "__main__":
    print(generate_restaurant_name_and_items("Indian")['restaurant_name'])
    print()
    print(generate_restaurant_name_and_items("Indian")['menu_items'])